##  Importing all the libraries

In [1]:
import os
%load_ext dotenv
%dotenv
!pip3 install prometheus-api-client

## Setting up the connection to scrape metrics from the Thanos instance
We need to obtain a session token in order to scrape the required Thanos instance on OpenShift. You can obtain this token value (after having logged in to OpenShift) by executing the following command in your local terminal: $oc whoami -t and setting it as an env variable.

## Collecting the metrics 


In [2]:
import prometheus_api_client as prom_api

In [3]:
url = os.environ.get("FLT_PROM_URL")
token = os.environ.get("FLT_PROM_ACCESS_TOKEN")

In [4]:
prom = prom_api.prometheus_connect.PrometheusConnect(url=url, headers={"Authorization": "bearer {}".format(token)}, disable_ssl=True)

In [5]:
all_metrics = prom.all_metrics()

/usr/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [6]:
all_metrics

['alerts',
 'cluster:capacity_cpu_cores:sum',
 'cluster:capacity_memory_bytes:sum',
 'cluster:cpu_usage_cores:sum',
 'cluster:memory_usage_bytes:sum',
 'cluster:node_instance_type_count:sum',
 'cluster_operator_conditions',
 'cluster_operator_up',
 'cluster_version',
 'cluster_version_available_updates',
 'cluster_version_payload',
 'code:apiserver_request_count:rate:sum',
 'etcd_object_counts',
 'kube_pod_status_ready:etcd:sum',
 'kube_pod_status_ready:image_registry:sum',
 'machine_cpu_cores',
 'machine_memory_bytes',
 'openshift:cpu_usage_cores:sum',
 'openshift:memory_usage_bytes:sum',
 'scrape_duration_seconds',
 'scrape_samples_post_metric_relabeling',
 'scrape_samples_scraped',
 'up']

In [7]:
print("Number of metrics present:", len(all_metrics))

Number of metrics present: 23


We can fetch the data for a specific metric name based on a particular time frame, duration size and label configurations

In [8]:
metric_data = prom.get_metric_range_data(metric_name='cluster_version', start_time='10m', chunk_size='5m')

/usr/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [9]:
len(metric_data)

1799

In [10]:
#Exploring one among all the metrics collected
metric_data[0].keys()

dict_keys(['metric', 'values'])

In [11]:
metadata = metric_data[0].get("metric")

In [12]:
metadata

{'__name__': 'cluster_version',
 '_id': '0073d69c-268c-4ec8-a01f-49afbac32d7b',
 'endpoint': 'metrics',
 'image': 'quay.io/openshift-release-dev/ocp-release@sha256:29a057c89cc5cf9c7853c52b815ba95bd66d1a66cb274d1991ca4951f5920cb2',
 'instance': '10.0.131.57:9099',
 'job': 'cluster-version-operator',
 'monitor': 'prometheus',
 'namespace': 'openshift-cluster-version',
 'pod': 'cluster-version-operator-66f586d9d-7bv6s',
 'prometheus': 'openshift-monitoring/k8s',
 'prometheus_replica': 'prometheus-telemeter-1',
 'replica': '$(HOSTNAME)',
 'service': 'cluster-version-operator',
 'type': 'cluster',
 'version': '4.0.0-0.9'}

In [13]:
#Obtaining the labels of the metric
labels = list(metadata.keys())
print("Number of labels identified:", len(labels))

Number of labels identified: 15


In [14]:
labels

['__name__',
 '_id',
 'endpoint',
 'image',
 'instance',
 'job',
 'monitor',
 'namespace',
 'pod',
 'prometheus',
 'prometheus_replica',
 'replica',
 'service',
 'type',
 'version']

In [15]:
#Obtaining the metric values
metric_values = metric_data[0].get('values')
metric_values

[[1555613915.201, '1555586845']]